# Import Statements

In [1]:
import pandas as pd
import numpy as np
import sqlite3
from matplotlib import pyplot as plt
import dominate
from dominate.tags import *
import espn_api.football
from espn_api.football import League
import time
import pickle
import uuid

from python import constants, functions

# Testing and Debugging

In [ ]:
def df_to_table2(
        data: pd.DataFrame,
        custom_columns: list[str] = None,
        row_id_columns: list[str] = None,
        table_id: str = None
) -> table:
    '''
    Function which converts a pandas DataFrame into an HTML table with optional styling.

    If dataframe is longer than 20 rows, table will be wrapped in a "scroll-table" div.

    Parameters
    ----------
    data : pd.DataFrame
        Dataframe to be converted.

    custom_columns : list[str], default None
        Columns to be displayed in the HTML table. If set to None, all columns in DataFrame will be used.

    row_id_columns : list[str], default None
        ID to be assigned to each tr tag in the tbody.
        Accepts a list of columns, will concatenate them starting with 'row' and separating with '-'.
        If set to None, no IDs will be assigned.

    table_id : str[str], default None
        ID to be assigned to the main table tag. If set to None, no ID will be given.

    Returns
    -------
    dominate.table or dominate.div
        Return a dominate HTML object. Can be either a table or div
    '''
    t = table()
    head = thead()
    body = tbody()

    records = data.to_dict('records')

    columns = data.columns
    if custom_columns:
        columns = custom_columns

    column_row = tr()
    for column in columns:
        column_row.add(th(column, _class=str(column).lower().replace(' ','-')))
    head.add(column_row)
    
    for record in records:
        data_row = tr(__pretty=False)
        if row_id_columns:
            row_id = 'row'
            for column in row_id_columns:
                row_id += ('-' + str(record[column]).lower())
            data_row['id'] = row_id

        for column in columns:
            data_row.add(td(record[column], _class=str(column).lower().replace(' ','-')))
        body.add(data_row)

    t.add(head)
    t.add(body)
    if table_id:
        t['id'] = table_id

    if len(data) > 20:
        scroll_div = div(_class='scroll-table')
        scroll_div.add(t)
        return scroll_div

    return t

In [ ]:
df = pd.DataFrame(
    [
        {'Team':'Haris', 'Week':1, 'Year':2019},
        {'Team':'George', 'Week':1, 'Year':2019},
        {'Team':'Kevin', 'Week':2, 'Year':2019},
        {'Team':'Dante', 'Week':2, 'Year':2019}
    ]
)

print(functions.df_to_table(df, table_id='test-table'))

<table id="test-table">
  <thead>
    <tr>
      <th>Team</th>
      <th>Week</th>
      <th>Year</th>
    </tr>
  </thead>
  <tbody>
    <tr><td>Haris</td><td>1</td><td>2019</td></tr>
    <tr><td>George</td><td>1</td><td>2019</td></tr>
    <tr><td>Kevin</td><td>2</td><td>2019</td></tr>
    <tr><td>Dante</td><td>2</td><td>2019</td></tr>
  </tbody>
</table>


In [6]:
year = 2023
team = 'Haris'

seasons = [functions.summary_table(constants.GAME_DATA, year=year) for year in constants.YEARS]
seasons_df = pd.concat(seasons)
seasons_df = seasons_df.loc[seasons_df['Team'] == team]
seasons_df
# print(functions.df_to_table(
#     data=standings,
#     custom_columns=['Team','Record','Ranking','Points For','Points Against','Avg Points For','Avg Margin','Luck Score'],
#     table_id='standings-table'
# ))

,Year,Team,Record,Ranking,Points For,Points Against,Avg Points For,Avg Margin,Luck Score
4,2019,Haris,7-6,5,1548.30,1483.26,119.10,5.00,1
9,2020,Haris,3-10,10,1440.82,1773.88,110.83,-25.62,-13
2,2021,Haris,8-6,3,1855.22,1655.12,132.52,14.29,4
6,2022,Haris,7-7,7,1647.48,1579.54,117.68,4.85,-3
0,2023,Haris,12-2,1,1801.34,1456.30,128.67,24.65,10
9,2024,Haris,3-11,10,1468.60,1616.88,104.90,-10.59,1
